In [1]:
import os
import cv2
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation,GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Set the path to your dataset
dataset_path = "/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train"

# Load the dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(128, 128),  # Resize images to a consistent size
    batch_size=32,          # Number of images per batch
    label_mode='categorical',  # or 'int' for numeric labels
    shuffle=True            # Shuffle the dataset
)

# Check the dataset
for images, labels in dataset.take(1):
    print(images.shape)  # (batch_size, height, width, channels)
    print(labels.shape)  # (batch_size, number_of_classes)

Found 18397 files belonging to 10 classes.
(32, 128, 128, 3)
(32, 10)


In [5]:
import tensorflow as tf

# Set the path to your dataset
dataset_path = "/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train"

# Load the training dataset (80%)
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(128, 128),        # Resize images
    batch_size=32,                # Number of images per batch
    validation_split=0.2,         # Reserve 20% for validation (or testing)
    subset='training',            # This is the training set
    seed=123,                     # Reproducibility
    shuffle=True                  # Shuffle the dataset
)

# Load the testing dataset (20%)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(128, 128),        # Resize images
    batch_size=32,                # Number of images per batch
    validation_split=0.2,         # Reserve 20% for validation (or testing)
    subset='validation',          # This is the validation/test set
    seed=123,                     # Reproducibility
    shuffle=True                  # Shuffle the dataset
)

# Check the datasets
for images, labels in train_dataset.take(1):
    print(f"Train: {images.shape}, {labels.shape}")

for images, labels in test_dataset.take(1):
    print(f"Test: {images.shape}, {labels.shape}")

Found 18397 files belonging to 10 classes.
Using 14718 files for training.
Found 18397 files belonging to 10 classes.
Using 3679 files for validation.
Train: (32, 128, 128, 3), (32,)
Test: (32, 128, 128, 3), (32,)


In [18]:
# Import the required modules
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50

# Create the base model from the pre-trained model ResNet50
base_model = ResNet50(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer with the number of classes in your dataset
predictions = Dense(10, activation='softmax')(x)  # Adjust the number of classes as needed

# This is the model we will train
resnet50_model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the ResNet50 layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model (should be done *after* setting layers to non-trainable)
resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
#Getting model's summary
resnet50_model.summary()

In [20]:
#Specifing epochs & batch size for vggnet16
epochs = 40
batch_size = 32
image_height = 128
image_width = 128

In [21]:
#Creating an object of ImageDataGenerator for augmenting training dataset
train_datagen = ImageDataGenerator(rescale= 1./255,
rotation_range=10,
width_shift_range=0.1,
height_shift_range=0.1,
shear_range=0.1,
zoom_range=0.1,
horizontal_flip=True,
fill_mode='nearest')

#Creating an object of ImageDataGenerator for augmenting test dataset
test_datagen = ImageDataGenerator(rescale= 1./255)

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
image_size = (128, 128)   # VGG19 requires 128*128 input
batch_size = 32           # Number of images to process in a batch

# Define the train data generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,           # Rescale pixel values to [0, 1]
    shear_range=0.2,          # Apply random shear transformations
    zoom_range=0.2,           # Apply random zoom
    horizontal_flip=True,      # Randomly flip images horizontally
    validation_split=0.2      # Split 20% of the data for validation
)

# Define the test/validation data generator (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Set dataset directory path
dataset_path = "/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train"

# Create train data generator
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,      # Resize all images to 128*128
    batch_size=batch_size,
    class_mode='categorical',    # For multi-class classification
    subset='training',           # Use this for the training data
    shuffle=True                 # Shuffle the data for better training
)

# Create validation data generator
validation_generator = test_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,      # Resize all images to 128*128
    batch_size=batch_size,
    class_mode='categorical',    # For multi-class classification
    subset='validation',         # Use this for the validation data
    shuffle=False                # No need to shuffle the validation data
)

# Check the generators
print(f"Train Generator: {len(train_generator)} batches")
print(f"Validation Generator: {len(validation_generator)} batches")


Found 14720 images belonging to 10 classes.
Found 3677 images belonging to 10 classes.
Train Generator: 460 batches
Validation Generator: 115 batches


In [24]:
# Define image size and batch size
image_size = (128, 128)
batch_size = 32           # Number of images to process in a batch

# Define the test data generator (only rescale, no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values to [0, 1]

# Set dataset directory path for test data
test_dataset_path = "/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train"

# Create test data generator
test_generator = test_datagen.flow_from_directory(
    test_dataset_path,
    target_size=image_size,      # Resize all images to 128*128
    batch_size=batch_size,
    class_mode='categorical',    # For multi-class classification
    shuffle=False                # Do not shuffle the test data
)

# Check the test generator
print(f"Test Generator: {len(test_generator)} batches")

Found 18397 images belonging to 10 classes.
Test Generator: 575 batches


In [25]:
resnet50_history = resnet50_model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/40


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


460/460 ━━━━━━━━━━━━━━━━━━━━ 10225s 22s/step - accuracy: 0.1491 - loss: 2.2674 - val_accuracy: 0.2711 - val_loss: 2.0082
Epoch 2/40
460/460 ━━━━━━━━━━━━━━━━━━━━ 1256s 3s/step - accuracy: 0.2699 - loss: 2.0287 - val_accuracy: 0.3497 - val_loss: 1.8739
Epoch 3/40
460/460 ━━━━━━━━━━━━━━━━━━━━ 1314s 3s/step - accuracy: 0.3474 - loss: 1.8330 - val_accuracy: 0.3919 - val_loss: 1.7305
Epoch 4/40
460/460 ━━━━━━━━━━━━━━━━━━━━ 1252s 3s/step - accuracy: 0.3912 - loss: 1.6948 - val_accuracy: 0.3908 - val_loss: 1.7489
Epoch 5/40
460/460 ━━━━━━━━━━━━━━━━━━━━ 1283s 3s/step - accuracy: 0.4107 - loss: 1.6400 - val_accuracy: 0.4115 - val_loss: 1.6262
Epoch 6/40
460/460 ━━━━━━━━━━━━━━━━━━━━ 1235s 3s/step - accuracy: 0.4381 - loss: 1.5776 - val_accuracy: 0.4283 - val_loss: 1.5805
Epoch 7/40
460/460 ━━━━━━━━━━━━━━━━━━━━ 1300s 3s/step - accuracy: 0.4493 - loss: 1.5395 - val_accuracy: 0.4115 - val_loss: 1.7039
Epoch 8/40
460/460 ━━━━━━━━━━━━━━━━━━━━ 1297s 3s/step - accuracy: 0.4781 - loss: 1.4734 - val_accur

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import classification_report

# predictions from the test data
y_pred = resnet50_model.predict(test_generator)

# transform class labels of the predictions
y_pred_classes = np.argmax(y_pred, axis=1)

# Real classes
y_true = test_generator.classes

# Create classification report
report = classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys())

# print report
print(report)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/575 ━━━━━━━━━━━━━━━━━━━━ 11:39 2s/step